In [1]:
import anndata
import numpy as np
import pandas as pd
import patsy

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [ ]:
exppd.read_csv("./data/bladderExp.csv",index_col=0)

In [ ]:
def num_sv(adata, model_matrix, method='be', vfilter=None, B=20, seed=None):
    if seed is not None:
        try:
            np.random.seed(seed)
        except:
            raise ValueError("seed should be integer")
    
    if vfilter is not None:
        if (vfilter < 100) or (vfilter > adata.shape[0]):
            raise ValueError("The number of genes used in the analysis must be between 100 and", str(adata.shape[0]),"\n")
        adata.obs['base_var'] = np.var(adata.X, ddof=1, axis=1)
        ind = np.where(np.linalg.matrix_rank(adata.X) < vfilter)
        adata = adata[adata.obs['base_var'][ind,]]
        